# PYTORCH CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/torch_cnn_classifier.ipynb torch_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
DATASET = 'no_dataset'
VERSION = 0
BATCH_SIZE = 32
EPOCHS = 10

In [2]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

human_nontata_promoters 0 32 2


## Config

In [11]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer

from genomic_benchmarks.dataset_getters.pytorch_datasets import get_dataset
from genomic_benchmarks.models.torch_cnn import CNN
from genomic_benchmarks.dataset_getters.utils import coll_factory, LetterTokenizer, build_vocab, check_seq_lengths, check_config, VARIABLE_LENGTH_DATASETS


In [4]:
USE_PADDING = DATASET in VARIABLE_LENGTH_DATASETS
    
config = {
    "dataset": DATASET,
    "dataset_version": VERSION,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "use_padding": USE_PADDING,
    "force_download": False,
    "run_on_gpu": True,
    "number_of_classes": 2,
    "embedding_dim": 100,
}
check_config(config)

## Choose the dataset

In [5]:
train_dset = get_dataset(config["dataset"], 'train')

/home/jupyter/genomic_benchmarks/src/genomic_benchmarks/loc2seq/with_biopython.py:96: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Unzipping...Done.


## Tokenizer and vocab

In [6]:
tokenizer = get_tokenizer(LetterTokenizer())
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 7
{'T': 4, '<eos>': 6, 'G': 3, 'C': 5, 'A': 2, '<bos>': 1, '<unk>': 0}


## Dataloader and batch preparation

In [7]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_seq_len, nn_input_len = check_seq_lengths(dataset=train_dset, config=config)

# Data Loader
if(config["use_padding"]):
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = nn_input_len)
else:
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = None)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cuda device
max_seq_len  251


## Model

In [8]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=nn_input_len
).to(device)

/opt/conda/envs/bench_env/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


## Training

In [9]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0
Train metrics: 
 Accuracy: 81.4%, Avg loss: 0.610314 

Epoch 1
Train metrics: 
 Accuracy: 81.6%, Avg loss: 0.609657 



## Testing

In [10]:
# test_dset = get_dataset_fn('test', force_download=config["force_download"], version=config["dataset_version"])
test_dset = get_dataset(config["dataset"], 'test')
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

model.test(test_loader)

Unzipping...Done.
test_loss  174.57336992025375
num_batches 283
correct 7242
size 9034
Test Error: 
 Accuracy: 80.2%, Avg loss: 0.616867 

